In [2]:
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
import re

# Loading the data

In [9]:
files = os.listdir('./001_medium')
texts = []
countries = []
positions = []
titles = []

for i in range(len(files) ):
#     print(i[:2])
#     print(files[i])
#     print( i, (files[i].split('.')[0]).split('_')[-1])
    # Read the text
    f = open("./001_medium/"+files[i], "r",  encoding="ISO-8859-1" )
    s = f.read()
    texts.append(s.strip())
    # Read the country
    countries.append(files[i].split('_')[0])
    # Read the position
    positions.append( (files[i].split('.')[0]).split('_')[-1] )
    # Read the full title
    titles.append( files[i].split('.') )
#     titles.append(files[i])
f.close()

In [10]:
# Creating a dataframe
df = pd.DataFrame(list(zip(countries, positions, texts)),
               columns =['country', 'title', 'text'])
df.head()

,country,title,text
0,AT,Inform,https://www.linkedin.com/company/global-blue/l...
1,AT,Architect,https://www.linkedin.com/jobs/view/2589036509/...
2,AT,Manager,www.linkedin.com/jobs/view/2540581439/\n\nInfo...
3,CZ,IPSecu,https://www.linkedin.com/jobs/view/2410398234/...
4,CZ,Senior,https://www.linkedin.com/jobs/view/2487448959/...


### Now I will write dataframe to csv so we don't have read files everytime

In [11]:
df.to_csv('jobs.csv', index=False)
df = pd.read_csv('jobs.csv')

In [23]:
df = pd.read_csv('jobs.csv')

## Basic feature extraction

In [13]:
df['word_count'] = df['text'].apply(lambda x: len( x.strip(' ')))
df[[ 'text', 'word_count']].head()

,text,word_count
0,https://www.linkedin.com/company/global-blue/l...,10293
1,https://www.linkedin.com/jobs/view/2589036509/...,2861
2,www.linkedin.com/jobs/view/2540581439/\n\nInfo...,4849
3,https://www.linkedin.com/jobs/view/2410398234/...,2673
4,https://www.linkedin.com/jobs/view/2487448959/...,3394


In [16]:
# Stop words per row
stop = stopwords.words('english')
df['stopwords'] = df['text'].apply(lambda x: len([x for x in x.split() if x in stop]))
df.head()

,country,title,text,word_count,stopwords
0,AT,Inform,https://www.linkedin.com/company/global-blue/l...,10293,300
1,AT,Architect,https://www.linkedin.com/jobs/view/2589036509/...,2861,134
2,AT,Manager,www.linkedin.com/jobs/view/2540581439/\n\nInfo...,4849,199
3,CZ,IPSecu,https://www.linkedin.com/jobs/view/2410398234/...,2673,82
4,CZ,Senior,https://www.linkedin.com/jobs/view/2487448959/...,3394,105


## Cleaning the text column

In [24]:
# Remove the links from text
df['text'] = df['text'].apply(lambda x: re.sub(r'https?://\S+|www\.\S+', '', x))
# Remove some additional unnecessary symbols from some rows and get job title position
df['job_title'] = df['text'].apply(lambda x: (re.sub(r'^.*?\n', '\n', x).strip()).split('\n')[0] )
df['text'] = df['text'].str.replace('[^\w\s]','')

df.head()

C:\Users\elsha\AppData\Local\Temp/ipykernel_11204/985002950.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]','')


,country,title,text,job_title
0,AT,Inform,\n\nInformation Technology Security Manager\n\...,Information Technology Security Manager
1,AT,Architect,\n\nInformation Security Architect\n\nCopmany ...,Information Security Architect
2,AT,Manager,\n\nInformation Technology Security Manager\n\...,Information Technology Security Manager
3,CZ,IPSecu,\n\n IP Security Architect\n\n\n About t...,IP Security Architect
4,CZ,Senior,\n \n Senior Information Security Analyst\...,Senior Information Security Analyst
